## SIMPD split
- **Approach:** For dataset with time series information,  dataset is sorted in chronological order based on a relevant time variable and then split into training and test sets. Importantly, the training set includes data from earlier time points, and the test set includes data from later time points.
- **Characteristics**: This approach mimics real-world scenarios where predictions are made on future data based on historical information. The model is trained on the training set, which consists of data up to a certain point in time. It is then evaluated on the test set, which contains data from a later time period. This provides a robust estimate of the model's ability to generalize to new data.
- **Application:** The assumption of using time-series split is that the key features of the dataset correlate with historical information. Most of the drug development projects requires a number of iterations of molecular design and experimental validation over a series of time points. The later design circles normally contain molecules having on one or multiple improved features of molecular profile, such as better range of molecular weight, new chemical structures,  new scaffolds, better bioactivity, better DMPK properties etc. In these cases, this  approach provides a robust estimate of the model's ability to **generalize to new data**. A model chosen through this splitting approach proves highly valuable in the lead optimization phase. However, assuming the molecular profiles have no significant changes over a different time point, applying the time-series split would have no difference to a random split. However, this method relies on the availability of the time series information.

In [1]:
%load_ext autoreload
%autoreload 2
import datamol as dm
from splito.simpd import SIMPDSplitter